# Kimore

In [1]:
import os
import csv
import openpyxl
import pandas as pd
import numpy as np

path = "Kimore"
kinect_joints = ["spinebase", "spinemid", "neck", "head", 
                 "shoulderleft", "elbowleft", "wristleft", 
                 "handleft", "shoulderright", "elbowright", 
                 "wristright", "handright", "hipleft", "kneeleft", 
                 "ankleleft", "footleft", "hipright", "kneeright", 
                 "ankleright", "footright", "spineshoulder", "handtipleft", 
                 "thumbleft", "handtipright", "thumbright"]

enable_kinect_joints = False
enable_slice_list = False

data = []
for (root, dirs, files) in os.walk(path):

    # if current directory contains "Raw", extract data
    if "Raw" in dirs:
        
        new_dict = {}
        
        # get exercise number
        new_dict["Exercise"] = int(root[-1])

        # extract raw data
        raw_files = os.listdir(os.path.join(root, "Raw"))
        for file in raw_files:

            file_path = os.path.join(os.path.join(root, "Raw"),file)
            csv_file = open(file_path, newline='')
            csv_reader = csv.reader(csv_file)
            
            if file.startswith("JointOrientation"):
                
                if enable_kinect_joints:
                    for joint in kinect_joints:
                        new_dict[joint + "-o"] = []

                    for row in csv_reader:
                        for i in range(len(kinect_joints)):
                            if len(row) > 0:
                                new_dict[kinect_joints[i] + "-o"].append(row[(4*i):(4*i+4)])
                else:
                    new_dict["JointOrientation"] = []
                    for row in csv_reader:
                        if len(new_dict["JointOrientation"]) >= 182 and enable_slice_list:
                            break
                        elif len(row) > 0:
                            if '' in row:
                                row.remove('')
                            new_dict["JointOrientation"].append(np.array([float(i) for i in row]))
                    np.array(new_dict["JointOrientation"])
                            

            elif file.startswith("JointPosition"):
                
                if enable_kinect_joints:
                    for joint in kinect_joints:
                        new_dict[joint + "-p"] = []

                    for row in csv_reader:
                        for i in range(len(kinect_joints)):
                            if len(row) > 0:
                                new_dict[kinect_joints[i] + "-p"].append(row[(4*i):(4*i+3)])
                else:
                    new_dict["JointPosition"] = []
                    for row in csv_reader:
                        if len(new_dict["JointPosition"]) >= 182 and enable_slice_list:
                            break
                        elif len(row) > 0:
                            if '' in row:
                                row.remove('')
                            new_dict["JointPosition"].append(np.array([float(i) for i in row]))
                    np.array(new_dict["JointPosition"])

            elif file.startswith("TimeStamp"):

                new_dict["Timestamps"] = []
                for row in csv_reader:
                    if len(new_dict["Timestamps"]) >= 182 and enable_slice_list:
                        break
                    elif len(row) > 0:
                        if '' in row:
                                row.remove('')
                        new_dict["Timestamps"].append(row)
                            
                        

        # extract data labels
        label_files = os.listdir(os.path.join(root, "Label"))
        for file in label_files:

            file_path = os.path.join(os.path.join(root, "Label"),file)
            book = openpyxl.load_workbook(file_path)
            sheet = book.active

            if file.startswith("SuppInfo"):
                for i in range(1, sheet.max_column):
                    t = sheet.cell(1, i).value
                    v = sheet.cell(2, i).value
                    new_dict[t] = v
            
            elif file.startswith("ClinicalAssessment"):
                new_dict["cTS"] = sheet.cell(2, new_dict["Exercise"]+1).value
                new_dict["cPO"] = sheet.cell(2, new_dict["Exercise"]+6).value
                new_dict["cCF"] = sheet.cell(2, new_dict["Exercise"]+11).value
        
        # append exercise to data
        data.append(new_dict)

C:\Users\Dilemre\anaconda3\lib\site-packages\openpyxl\reader\excel.py:228: UserWarning: Unknown extension is not supported and will be removed
  ws_parser.bind_all()
C:\Users\Dilemre\AppData\Local\Temp\ipykernel_8636\2278027947.py:57: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(new_dict["JointOrientation"])
C:\Users\Dilemre\AppData\Local\Temp\ipykernel_8636\2278027947.py:79: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(new_dict["JointPosition"])


In [2]:
def findOutlierLen(JP, JO):
    """
    Parameters:
        JP: JointPosition.
        JO: JointOrientation.
    """
    signal = 1
    outlier = []
    for i in range(0, len(JP)):
        for j in range(0, len(JP[i])):
            if len(JP[i][j]) != 100:
                signal = 0
                outlier.append(i)
                break
        
    if signal:
        print("No outlier lenght.")
        
    return outlier
      
    
def isSameShape(JP, JO):
    """
    Parameters:
        JP: JointPosition.
        JO: JointOrientation.
    """
    signal = 1
    for i in range(0, len(JP)):
        for j in range(0, len(JP[i])):
            if np.shape(JP) != np.shape(JO):
                signal = 0
                print("Position:\t", i, j, "len: ",len(JP[i][j]))
                print("Orientation:\t", i, j, "len: ",len(JO[i][j]))

    if signal:
        print("All equally shaped.")

        
def minMaxLen(TM):
    """
    Parameters:
        TM: Timestamps.
    """
    print("min len: ", TM.apply(len).sort_values().iat[0], "idx: ", TM.apply(len).sort_values().index[0],
      "\nmax len: ", TM.apply(len).sort_values(ascending=False).iat[0], "idx: ", TM.apply(len).sort_values(ascending=False).index[0])
    
    
def groupExerciseCount(df):
    """
    Parameters:
        df: Dataframe
    """
    for g in df.Group.unique():
        for e in df.Exercise.unique():
            print("Group:", g, "\tExercise:", e, "\tCount:", df[(df["Group"]==g) & (df["Exercise"]==e)].shape[0])

In [3]:
df = pd.DataFrame(data)

In [4]:
df

,Exercise,JointOrientation,JointPosition,Timestamps,cTS,cPO,cCF,Subject ID,Group,Age
0,1,"[[-0.0531085, 0.00232358, 0.99423, -0.0931733,...","[[-0.0468451, -0.409343, 2.6241, 2.0, -0.04849...","[[20622112895], [20622442942], [20622783803], ...",48.333333,14.666667,33.666667,E_ID1,E,50
1,2,"[[-0.0610258, 0.00258101, 0.995022, -0.0787486...","[[-0.0578279, -0.407365, 2.60017, 2.0, -0.0592...","[[21184779662], [21185108840], [21185438830], ...",45.000000,13.666667,31.333333,E_ID1,E,50
2,3,"[[-0.0298744, -0.00814489, 0.995711, -0.087179...","[[-0.0652774, -0.418544, 2.56966, 2.0, -0.0718...","[[21902433786], [21902774551], [21903104529], ...",48.333333,15.000000,33.333333,E_ID1,CG_E,50
3,4,"[[-0.0369213, -0.00490366, 0.995614, -0.085821...","[[-0.0674865, -0.397115, 2.58841, 2.0, -0.0723...","[[22621099365], [22621428603], [22621769446], ...",50.000000,15.000000,35.000000,E_ID1,CG_E,50
4,5,"[[-0.0517414, -0.000435555, 0.99603, -0.072440...","[[-0.0630149, -0.394823, 2.59342, 2.0, -0.0655...","[[23248424127], [23248764907], [23249094898], ...",48.333333,15.000000,33.333333,E_ID1,CG_E,50
...,...,...,...,...,...,...,...,...,...,...
385,1,"[[0.039838, -0.0277761, 0.995785, -0.0777996, ...","[[-0.20825, -0.500367, 2.32432, 2.0, -0.223546...","[[711302136484], [711302477267], [711302806451...",26.333333,6.333333,20.000000,S_ID9,S,67
386,2,"[[0.0600943, -0.0332151, 0.995777, -0.0609322,...","[[-0.209425, -0.48988, 2.31622, 2.0, -0.227487...","[[711894802395], [711895132382], [711895473180...",17.666667,4.333333,13.333333,S_ID9,S,67
387,3,"[[0.0795781, -0.0310581, 0.994218, -0.0650627,...","[[-0.202571, -0.492478, 2.31694, 2.0, -0.21846...","[[712329470249], [712329799436], [712330129378...",26.333333,6.333333,20.000000,S_ID9,S,67
388,4,"[[-0.00995722, -0.0160432, 0.997548, -0.067393...","[[-0.199238, -0.471849, 2.29448, 2.0, -0.20909...","[[712712796766], [712713126759], [712713467555...",23.000000,7.000000,16.000000,S_ID9,S,67


In [5]:
df.at[2, 'Group'] = "E"
df.at[3, 'Group'] = "E"
df.at[4, 'Group'] = "E"

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Exercise,390.0,3.000000,1.416030,1.000000,2.000000,3.000000,4.0,5.0
cTS,383.0,37.457573,10.204780,10.000000,30.140452,40.000000,46.0,50.0
cPO,383.0,11.040233,3.952683,0.005147,8.540081,12.000000,14.5,15.0
cCF,383.0,26.413858,7.043139,7.000000,22.276553,27.360633,32.0,35.0
Age,390.0,45.641026,19.354609,17.000000,26.000000,45.500000,63.0,81.0


In [7]:
df.isnull().sum()

Exercise            0
JointOrientation    5
JointPosition       5
Timestamps          3
cTS                 7
cPO                 7
cCF                 7
Subject ID          0
Group               0
Age                 0
dtype: int64

In [8]:
dropped_rows = df[df.isna().any(axis=1)]
dropped_rows

,Exercise,JointOrientation,JointPosition,Timestamps,cTS,cPO,cCF,Subject ID,Group,Age
40,1,"[[-0.0481282, 8.87374e-07, 0.985768, 0.161073,...","[[0.175871, -1.10702, 2.84095, 2.0, 0.180718, ...","[[30437599246420], [30437599576380], [30437599...",NaN,NaN,NaN,E_ID17,E,45
41,2,"[[0.0199295, -0.0368523, 0.996205, 0.0762919, ...","[[0.155424, -1.09849, 3.29907, 2.0, 0.13016, -...","[[30438448240198], [30438448570198], [30438448...",NaN,NaN,NaN,E_ID17,E,45
42,3,"[[-0.228232, 0.030343, 0.964276, 0.131002, -0....","[[0.173645, -1.03028, 3.27985, 2.0, 0.209115, ...","[[30439025566426], [30439025896688], [30439026...",NaN,NaN,NaN,E_ID17,E,45
43,4,"[[-0.101832, 0.0088296, 0.98639, 0.128791, -0....","[[0.277114, -1.03271, 3.01552, 2.0, 0.290556, ...","[[30442081214560], [30442081543751], [30442081...",NaN,NaN,NaN,E_ID17,E,45
44,5,"[[-0.0201885, -0.0603588, 0.992337, -0.105904,...","[[0.300576, -1.3918, 3.14916, 2.0, 0.263258, -...","[[30442714879944], [30442715209965], [30442715...",NaN,NaN,NaN,E_ID17,E,45
76,2,NaN,NaN,NaN,NaN,NaN,NaN,E_ID8,E,22
97,3,NaN,"[[-0.196504, -0.761454, 2.62517, 2.0, -0.19706...","[[10369220941503], [10369221270654], [10369221...",40.00000,15.000000,25.000000,NE_ID11,NE,75
99,5,[],NaN,"[[10370694259997], [10370694589964], [10370694...",49.00000,14.000000,35.000000,NE_ID11,NE,75
166,2,"[[0.0367658, 0.00355401, 0.998187, -0.0475269,...",NaN,"[[184099250059], [184099580040], [184099910803...",44.00000,13.000000,31.000000,NE_ID24,NE,25
258,4,NaN,NaN,NaN,NaN,NaN,NaN,B_ID8,B,36


In [9]:
del dropped_rows

In [10]:
df = df.dropna().reset_index(drop=True)

In [11]:
outlier = findOutlierLen(df.JointPosition, df.JointOrientation)
if len(outlier) > 0:
    df = df.drop(outlier).reset_index(drop=True)
    outlier = findOutlierLen(df.JointPosition, df.JointOrientation)
del outlier

No outlier lenght.


In [12]:
isSameShape(df.JointPosition, df.JointOrientation)

All equally shaped.


In [13]:
minMaxLen(df.Timestamps)

min len:  182 idx:  26 
max len:  2487 idx:  327


In [14]:
groupExerciseCount(df)

Group: E 	Exercise: 1 	Count: 15
Group: E 	Exercise: 2 	Count: 15
Group: E 	Exercise: 3 	Count: 16
Group: E 	Exercise: 4 	Count: 16
Group: E 	Exercise: 5 	Count: 16
Group: NE 	Exercise: 1 	Count: 27
Group: NE 	Exercise: 2 	Count: 26
Group: NE 	Exercise: 3 	Count: 26
Group: NE 	Exercise: 4 	Count: 27
Group: NE 	Exercise: 5 	Count: 26
Group: B 	Exercise: 1 	Count: 8
Group: B 	Exercise: 2 	Count: 8
Group: B 	Exercise: 3 	Count: 8
Group: B 	Exercise: 4 	Count: 7
Group: B 	Exercise: 5 	Count: 8
Group: P 	Exercise: 1 	Count: 15
Group: P 	Exercise: 2 	Count: 16
Group: P 	Exercise: 3 	Count: 16
Group: P 	Exercise: 4 	Count: 16
Group: P 	Exercise: 5 	Count: 15
Group: S 	Exercise: 1 	Count: 10
Group: S 	Exercise: 2 	Count: 10
Group: S 	Exercise: 3 	Count: 9
Group: S 	Exercise: 4 	Count: 10
Group: S 	Exercise: 5 	Count: 9


In [15]:
from numpy.lib.stride_tricks import sliding_window_view

def windowing(joints, shape = 100):
    v = sliding_window_view(np.array(joints[0:len(joints) - (len(joints) % 100)]).flatten(), shape)
    return v[::v.shape[0]//500,:]

In [16]:
df.JointPosition = df.JointPosition.apply(lambda x: windowing(x)[:500, :]).to_numpy()
df.JointOrientation = df.JointOrientation.apply(lambda x: windowing(x)[:500, :]).to_numpy()

In [17]:
list_of_group = []
for g in df.Group.unique():
    for e in df.Exercise.unique():
        subset = df.loc[(df["Group"] == g) & (df["Exercise"] == e), :].sample(7)
        list_of_group.append(subset)
        
downsampled_df = pd.concat(list_of_group, ignore_index=True)
del list_of_group

In [18]:
downsampled_df = downsampled_df.drop(columns=["Timestamps", "cTS", "cPO", "cCF",  "Age "])
e = downsampled_df.pop("Exercise")
g = downsampled_df.pop("Group")
downsampled_df["Exercise"] = e
downsampled_df["Group"] = g
del e
del g
downsampled_df

,JointOrientation,JointPosition,Subject ID,Exercise,Group
0,"[[-0.0531085, 0.00232358, 0.99423, -0.0931733,...","[[-0.0468451, -0.409343, 2.6241, 2.0, -0.04849...",E_ID1,1,E
1,"[[-0.0020808, -0.00690942, 0.997573, -0.069259...","[[-0.120613, -0.255417, 2.52227, 2.0, -0.12552...",E_ID11,1,E
2,"[[-0.00870099, 3.66002e-06, 0.994934, -0.10015...","[[-0.0420686, -0.293024, 2.74052, 2.0, -0.0426...",E_ID3,1,E
3,"[[-0.00766952, -0.0174588, 0.995782, -0.089742...","[[-0.167507, -0.427232, 2.35279, 2.0, -0.17801...",E_ID10,1,E
4,"[[-0.0179779, 0.00536922, 0.996446, -0.0821169...","[[-0.0811335, -0.28634, 2.31876, 2.0, -0.07840...",E_ID6,1,E
...,...,...,...,...,...
170,"[[0.057045, -0.0307528, 0.996105, -0.0597952, ...","[[-0.177143, -0.506639, 2.30473, 2.0, -0.19405...",S_ID9,5,S
171,"[[0.0180271, 0.0140138, 0.996382, -0.0818583, ...","[[-0.355986, -0.549952, 2.51772, 2.0, -0.34697...",S_ID10,5,S
172,"[[0.0037668, -0.158592, 0.945976, -0.282777, -...","[[-0.312456, -0.214189, 2.88801, 2.0, -0.33213...",S_ID3,5,S
173,"[[0.0994289, -0.0135622, 0.993961, -0.0443926,...","[[-0.551777, -0.653763, 3.37547, 2.0, -0.55710...",S_ID1,5,S


In [19]:
groupExerciseCount(downsampled_df)

Group: E 	Exercise: 1 	Count: 7
Group: E 	Exercise: 2 	Count: 7
Group: E 	Exercise: 3 	Count: 7
Group: E 	Exercise: 4 	Count: 7
Group: E 	Exercise: 5 	Count: 7
Group: NE 	Exercise: 1 	Count: 7
Group: NE 	Exercise: 2 	Count: 7
Group: NE 	Exercise: 3 	Count: 7
Group: NE 	Exercise: 4 	Count: 7
Group: NE 	Exercise: 5 	Count: 7
Group: B 	Exercise: 1 	Count: 7
Group: B 	Exercise: 2 	Count: 7
Group: B 	Exercise: 3 	Count: 7
Group: B 	Exercise: 4 	Count: 7
Group: B 	Exercise: 5 	Count: 7
Group: P 	Exercise: 1 	Count: 7
Group: P 	Exercise: 2 	Count: 7
Group: P 	Exercise: 3 	Count: 7
Group: P 	Exercise: 4 	Count: 7
Group: P 	Exercise: 5 	Count: 7
Group: S 	Exercise: 1 	Count: 7
Group: S 	Exercise: 2 	Count: 7
Group: S 	Exercise: 3 	Count: 7
Group: S 	Exercise: 4 	Count: 7
Group: S 	Exercise: 5 	Count: 7


In [20]:
minMaxLen(downsampled_df.JointOrientation)

min len:  500 idx:  0 
max len:  500 idx:  0


In [21]:
def slice_joints(df):
    temp = pd.DataFrame()
    temp["JointOrientation"] = np.array_split(df.JointOrientation, 10)
    temp["JointPosition"] = np.array_split(df.JointPosition, 10)
    temp["Subject ID"] = np.repeat(df["Subject ID"], 10)
    temp["Exercise"] = np.repeat(df.Exercise, 10)
    temp["Group"] = np.repeat(df.Group, 10)
    
    return temp

In [22]:
list_of_rows = []
for _, row in downsampled_df.iterrows():
    list_of_rows.append(slice_joints(row))
new_df = pd.concat(list_of_rows, ignore_index=True)
del list_of_rows
del downsampled_df
new_df

,JointOrientation,JointPosition,Subject ID,Exercise,Group
0,"[[-0.0531085, 0.00232358, 0.99423, -0.0931733,...","[[-0.0468451, -0.409343, 2.6241, 2.0, -0.04849...",E_ID1,1,E
1,"[[0.724844, -0.194065, -0.0120182, -0.672086, ...","[[2.57944, 2.0, -0.166502, -0.691469, 2.63628,...",E_ID1,1,E
2,"[[-0.045894, 0.00228192, 0.994775, -0.0911662,...","[[-0.0478889, -0.402169, 2.61568, 2.0, -0.0490...",E_ID1,1,E
3,"[[0.716875, -0.195914, -0.00942156, -0.671347,...","[[2.58492, 2.0, -0.169192, -0.686221, 2.64052,...",E_ID1,1,E
4,"[[-0.0557715, -0.00244316, 0.994198, -0.091950...","[[-0.0491798, -0.403752, 2.61994, 2.0, -0.0539...",E_ID1,1,E
...,...,...,...,...,...
1745,"[[-0.680607, 0.116872, 0.097614, -0.312216, 0....","[[3.33574, 2.0, -0.271246, -1.00124, 3.34321, ...",S_ID4,5,S
1746,"[[0.0130074, -0.00649919, 0.988356, -0.151465,...","[[-0.12987, -0.927107, 3.47808, 2.0, -0.132292...",S_ID4,5,S
1747,"[[0.683068, -0.122742, 0.0371399, 0.99561, -0....","[[3.31464, 2.0, -0.250759, -1.00152, 3.33506, ...",S_ID4,5,S
1748,"[[0.0424746, -0.0268141, 0.993122, -0.105767, ...","[[-0.146948, -0.867697, 3.46373, 2.0, -0.15920...",S_ID4,5,S


In [23]:
outlier = findOutlierLen(new_df.JointPosition, new_df.JointOrientation)
del outlier

No outlier lenght.


In [24]:
isSameShape(new_df.JointPosition, new_df.JointOrientation)

All equally shaped.


In [25]:
minMaxLen(new_df.JointPosition)

min len:  50 idx:  0 
max len:  50 idx:  0


In [26]:
groupExerciseCount(new_df)

Group: E 	Exercise: 1 	Count: 70
Group: E 	Exercise: 2 	Count: 70
Group: E 	Exercise: 3 	Count: 70
Group: E 	Exercise: 4 	Count: 70
Group: E 	Exercise: 5 	Count: 70
Group: NE 	Exercise: 1 	Count: 70
Group: NE 	Exercise: 2 	Count: 70
Group: NE 	Exercise: 3 	Count: 70
Group: NE 	Exercise: 4 	Count: 70
Group: NE 	Exercise: 5 	Count: 70
Group: B 	Exercise: 1 	Count: 70
Group: B 	Exercise: 2 	Count: 70
Group: B 	Exercise: 3 	Count: 70
Group: B 	Exercise: 4 	Count: 70
Group: B 	Exercise: 5 	Count: 70
Group: P 	Exercise: 1 	Count: 70
Group: P 	Exercise: 2 	Count: 70
Group: P 	Exercise: 3 	Count: 70
Group: P 	Exercise: 4 	Count: 70
Group: P 	Exercise: 5 	Count: 70
Group: S 	Exercise: 1 	Count: 70
Group: S 	Exercise: 2 	Count: 70
Group: S 	Exercise: 3 	Count: 70
Group: S 	Exercise: 4 	Count: 70
Group: S 	Exercise: 5 	Count: 70


In [27]:
# y = (downsampled_df.cTS*2/100)
# print("Max value: ", y.max(), "Original value: ", downsampled_df.cTS.iat[y.idxmax()], "idx: ", y.idxmax(),
#       "\nMin value: ", y.min(),  "Original value: ", downsampled_df.cTS.iat[y.idxmin()], "idx: ", y.idxmin())

In [28]:
x1 = np.array(list(zip(new_df.JointOrientation, new_df.JointPosition)))

In [29]:
x1.shape

(1750, 2, 50, 100)

In [30]:
x1 = np.transpose(x1, (0, 2, 3, 1))
x1.shape

(1750, 50, 100, 2)

In [31]:
x2 = np.concatenate([np.stack(new_df.JointOrientation),np.stack(new_df.JointPosition)], axis=-1)
x2.shape

(1750, 50, 200)

In [36]:
y = new_df.Exercise.to_numpy()
y -=1
y.shape

(1750,)

In [37]:
def get_dataset_partitions(x, y, train_split=0.8, val_split=0.1, test_split=0.1):
    assert (train_split + test_split + val_split) == 1
    
    # Only allows for equal validation and test splits
    # assert val_split == test_split 
    
    assert len(x) == len(y)
    # Specify seed to always have the same split distribution between runs
    p = np.random.permutation(len(y))
    indices_or_sections = [int(train_split * len(y)), int((1 - val_split - test_split) * len(y))]
    
    train_x, val_x, test_x = np.split(x[p], indices_or_sections)
    train_y, val_y, test_y = np.split(y[p], indices_or_sections)
    
    return train_x, val_x, test_x, train_y, val_y, test_y

train_x, val_x, test_x, train_y, val_y, test_y = get_dataset_partitions(x1, y, val_split = 0, test_split = 0.2)

In [38]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import Sequential

In [39]:
input_shape = (50, 100, 2)

In [45]:
model = Sequential()
model.add(Conv2D(16, (2, 2), activation="relu", padding="same", input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Reshape((200, 100)))
model.add(LSTM(16, activation="tanh"))
model.add(Dense(5))

In [46]:
model.build(input_shape=input_shape)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 50, 100, 16)       144       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 50, 16)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 20000)             0         
                                                                 
 reshape_1 (Reshape)         (None, 200, 100)          0         
                                                                 
 lstm_1 (LSTM)               (None, 16)                7488      
                                                                 
 dense_1 (Dense)             (None, 5)                 85        
                                                      

In [47]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [48]:
history = model.fit(train_x, train_y, epochs=100, validation_split=0.2)

Epoch 1/100
35/35 [==============================] - 5s 91ms/step - loss: 1.6298 - accuracy: 0.1652 - val_loss: 1.6129 - val_accuracy: 0.2000
Epoch 2/100
35/35 [==============================] - 3s 78ms/step - loss: 1.6051 - accuracy: 0.2277 - val_loss: 1.5995 - val_accuracy: 0.2571
Epoch 3/100
35/35 [==============================] - 3s 78ms/step - loss: 1.5959 - accuracy: 0.2580 - val_loss: 1.5893 - val_accuracy: 0.2786
Epoch 4/100
35/35 [==============================] - 3s 81ms/step - loss: 1.5842 - accuracy: 0.2795 - val_loss: 1.5717 - val_accuracy: 0.2893
Epoch 5/100
35/35 [==============================] - 3s 79ms/step - loss: 1.5616 - accuracy: 0.3027 - val_loss: 1.5465 - val_accuracy: 0.2857
Epoch 6/100
35/35 [==============================] - 3s 78ms/step - loss: 1.5338 - accuracy: 0.3098 - val_loss: 1.4863 - val_accuracy: 0.3571
Epoch 7/100
35/35 [==============================] - 3s 78ms/step - loss: 1.4883 - accuracy: 0.3473 - val_loss: 1.4452 - val_accuracy: 0.4179
Epoch 

35/35 [==============================] - 3s 72ms/step - loss: 0.6608 - accuracy: 0.7509 - val_loss: 0.9611 - val_accuracy: 0.6393
Epoch 59/100
35/35 [==============================] - 2s 70ms/step - loss: 0.6293 - accuracy: 0.7598 - val_loss: 1.1393 - val_accuracy: 0.6179
Epoch 60/100
35/35 [==============================] - 2s 71ms/step - loss: 0.6505 - accuracy: 0.7500 - val_loss: 0.9651 - val_accuracy: 0.6286
Epoch 61/100
35/35 [==============================] - 2s 72ms/step - loss: 0.6095 - accuracy: 0.7661 - val_loss: 0.9236 - val_accuracy: 0.6786
Epoch 62/100
35/35 [==============================] - 2s 72ms/step - loss: 0.6005 - accuracy: 0.7652 - val_loss: 0.9643 - val_accuracy: 0.6536
Epoch 63/100
35/35 [==============================] - 2s 70ms/step - loss: 0.5864 - accuracy: 0.7750 - val_loss: 1.0752 - val_accuracy: 0.6214
Epoch 64/100
35/35 [==============================] - 2s 71ms/step - loss: 0.6021 - accuracy: 0.7705 - val_loss: 0.9554 - val_accuracy: 0.6643
Epoch 65/100

In [49]:
 model.evaluate(test_x, test_y)

11/11 [==============================] - 0s 18ms/step - loss: 0.9505 - accuracy: 0.6800


[0.9505482912063599, 0.6800000071525574]

In [66]:
model2 = Sequential()
model2.add(Conv2D(32, (2, 2), activation="relu", padding="same", input_shape=input_shape))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Flatten())
model2.add(Reshape((400, 100)))
model2.add(LSTM(32, activation="softmax"))
model2.add(Dense(5))
model2.build(input_shape=input_shape)
model2.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 50, 100, 32)       288       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 25, 50, 32)       0         
 2D)                                                             
                                                                 
 flatten_9 (Flatten)         (None, 40000)             0         
                                                                 
 reshape_9 (Reshape)         (None, 400, 100)          0         
                                                                 
 lstm_10 (LSTM)              (None, 32)                17024     
                                                                 
 dense_6 (Dense)             (None, 5)                 165       
                                                      

In [67]:
model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [68]:
history2 = model2.fit(train_x, train_y, epochs=100, validation_split=0.2)

Epoch 1/100
35/35 [==============================] - 8s 201ms/step - loss: 1.6099 - accuracy: 0.1866 - val_loss: 1.6080 - val_accuracy: 0.2071
Epoch 2/100
35/35 [==============================] - 6s 184ms/step - loss: 1.6092 - accuracy: 0.2116 - val_loss: 1.6082 - val_accuracy: 0.2071
Epoch 3/100
35/35 [==============================] - 7s 193ms/step - loss: 1.6087 - accuracy: 0.2116 - val_loss: 1.6086 - val_accuracy: 0.2071
Epoch 4/100
35/35 [==============================] - 7s 210ms/step - loss: 1.6083 - accuracy: 0.2116 - val_loss: 1.6080 - val_accuracy: 0.2179
Epoch 5/100
35/35 [==============================] - 8s 234ms/step - loss: 1.6056 - accuracy: 0.2330 - val_loss: 1.6047 - val_accuracy: 0.2321
Epoch 6/100
35/35 [==============================] - 7s 204ms/step - loss: 1.6059 - accuracy: 0.2188 - val_loss: 1.6061 - val_accuracy: 0.2107
Epoch 7/100
35/35 [==============================] - 6s 177ms/step - loss: 1.6027 - accuracy: 0.2179 - val_loss: 1.6018 - val_accuracy: 0.2321

Epoch 58/100
35/35 [==============================] - 7s 188ms/step - loss: 1.2602 - accuracy: 0.4679 - val_loss: 1.2790 - val_accuracy: 0.4714
Epoch 59/100
35/35 [==============================] - 7s 192ms/step - loss: 1.2731 - accuracy: 0.4741 - val_loss: 1.2727 - val_accuracy: 0.5321
Epoch 60/100
35/35 [==============================] - 7s 189ms/step - loss: 1.2482 - accuracy: 0.4911 - val_loss: 1.3118 - val_accuracy: 0.4607
Epoch 61/100
35/35 [==============================] - 7s 195ms/step - loss: 1.2547 - accuracy: 0.4902 - val_loss: 1.2694 - val_accuracy: 0.4929
Epoch 62/100
35/35 [==============================] - 7s 191ms/step - loss: 1.2540 - accuracy: 0.4812 - val_loss: 1.2620 - val_accuracy: 0.4929
Epoch 63/100
35/35 [==============================] - 7s 190ms/step - loss: 1.2475 - accuracy: 0.5063 - val_loss: 1.2548 - val_accuracy: 0.5250
Epoch 64/100
35/35 [==============================] - 7s 190ms/step - loss: 1.2382 - accuracy: 0.5312 - val_loss: 1.2868 - val_accuracy:

In [69]:
 model2.evaluate(test_x, test_y)

11/11 [==============================] - 1s 58ms/step - loss: 1.1965 - accuracy: 0.5171


[1.1964865922927856, 0.5171428322792053]

In [77]:
model3 = Sequential()
model3.add(Conv2D(32, (2, 2), activation="relu", padding="same", input_shape=input_shape))
model3.add(MaxPooling2D(pool_size=(2, 2)))
model3.add(Flatten())
model3.add(Reshape((400, 100)))
model3.add(LSTM(32, activation="tanh"))
model3.add(Dense(5, activation="softmax"))
model3.build(input_shape=input_shape)
model3.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_13 (Conv2D)          (None, 50, 100, 32)       288       
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 25, 50, 32)       0         
 g2D)                                                            
                                                                 
 flatten_13 (Flatten)        (None, 40000)             0         
                                                                 
 reshape_13 (Reshape)        (None, 400, 100)          0         
                                                                 
 lstm_14 (LSTM)              (None, 32)                17024     
                                                                 
 dense_10 (Dense)            (None, 5)                 165       
                                                     

In [80]:
model3.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [81]:
history3 = model3.fit(train_x, train_y, epochs=100, validation_split=0.2)

Epoch 1/100
35/35 [==============================] - 8s 185ms/step - loss: 1.6138 - accuracy: 0.1973 - val_loss: 1.6113 - val_accuracy: 0.1929
Epoch 2/100
35/35 [==============================] - 6s 175ms/step - loss: 1.5987 - accuracy: 0.2170 - val_loss: 1.5918 - val_accuracy: 0.2786
Epoch 3/100
35/35 [==============================] - 7s 190ms/step - loss: 1.5912 - accuracy: 0.2455 - val_loss: 1.5745 - val_accuracy: 0.3179
Epoch 4/100
35/35 [==============================] - 6s 183ms/step - loss: 1.5711 - accuracy: 0.3152 - val_loss: 1.5617 - val_accuracy: 0.2857
Epoch 5/100
35/35 [==============================] - 7s 187ms/step - loss: 1.5373 - accuracy: 0.3187 - val_loss: 1.5029 - val_accuracy: 0.3750
Epoch 6/100
35/35 [==============================] - 7s 187ms/step - loss: 1.4835 - accuracy: 0.3500 - val_loss: 1.4569 - val_accuracy: 0.3786
Epoch 7/100
35/35 [==============================] - 7s 186ms/step - loss: 1.4309 - accuracy: 0.3982 - val_loss: 1.3322 - val_accuracy: 0.4893

Epoch 58/100
35/35 [==============================] - 7s 198ms/step - loss: 0.7307 - accuracy: 0.7054 - val_loss: 0.9664 - val_accuracy: 0.6357
Epoch 59/100
35/35 [==============================] - 6s 176ms/step - loss: 0.6719 - accuracy: 0.7563 - val_loss: 0.9574 - val_accuracy: 0.6071
Epoch 60/100
35/35 [==============================] - 6s 171ms/step - loss: 0.6893 - accuracy: 0.7321 - val_loss: 0.9002 - val_accuracy: 0.6500
Epoch 61/100
35/35 [==============================] - 6s 170ms/step - loss: 0.6798 - accuracy: 0.7348 - val_loss: 0.9108 - val_accuracy: 0.6357
Epoch 62/100
35/35 [==============================] - 6s 171ms/step - loss: 0.6350 - accuracy: 0.7500 - val_loss: 0.8952 - val_accuracy: 0.6536
Epoch 63/100
35/35 [==============================] - 6s 172ms/step - loss: 0.6610 - accuracy: 0.7509 - val_loss: 0.8731 - val_accuracy: 0.6750
Epoch 64/100
35/35 [==============================] - 6s 185ms/step - loss: 0.6438 - accuracy: 0.7518 - val_loss: 0.9141 - val_accuracy:

In [82]:
 model3.evaluate(test_x, test_y)

11/11 [==============================] - 1s 49ms/step - loss: 1.0917 - accuracy: 0.6829


[1.0917373895645142, 0.6828571557998657]

In [153]:
model4 = Sequential()
model4.add(Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=input_shape))
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(Conv2D(64, (3, 3), activation="relu", padding="same", input_shape=input_shape))
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(Flatten())
model4.add(Reshape((-1, 64)))
model4.add(LSTM(32, activation="tanh"))
model4.add(Dense(5, activation="softmax"))
model4.build(input_shape=input_shape)
model4.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_27 (Conv2D)          (None, 50, 100, 32)       608       
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 25, 50, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_28 (Conv2D)          (None, 25, 50, 64)        18496     
                                                                 
 max_pooling2d_28 (MaxPoolin  (None, 12, 25, 64)       0         
 g2D)                                                            
                                                                 
 flatten_23 (Flatten)        (None, 19200)             0         
                                                                 
 reshape_22 (Reshape)        (None, 300, 64)         

In [154]:
model4.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [155]:
history4 = model4.fit(train_x, train_y, epochs=100, validation_split=0.2)

Epoch 1/100
35/35 [==============================] - 9s 206ms/step - loss: 1.6162 - accuracy: 0.2152 - val_loss: 1.6069 - val_accuracy: 0.2714
Epoch 2/100
35/35 [==============================] - 7s 192ms/step - loss: 1.5827 - accuracy: 0.2839 - val_loss: 1.4914 - val_accuracy: 0.4214
Epoch 3/100
35/35 [==============================] - 6s 185ms/step - loss: 1.4909 - accuracy: 0.3420 - val_loss: 1.4309 - val_accuracy: 0.3714
Epoch 4/100
35/35 [==============================] - 6s 179ms/step - loss: 1.4050 - accuracy: 0.3866 - val_loss: 1.3181 - val_accuracy: 0.4536
Epoch 5/100
35/35 [==============================] - 6s 178ms/step - loss: 1.2870 - accuracy: 0.4527 - val_loss: 1.2471 - val_accuracy: 0.4821
Epoch 6/100
35/35 [==============================] - 6s 169ms/step - loss: 1.2638 - accuracy: 0.4554 - val_loss: 1.2542 - val_accuracy: 0.4857
Epoch 7/100
35/35 [==============================] - 6s 181ms/step - loss: 1.4305 - accuracy: 0.4098 - val_loss: 1.7645 - val_accuracy: 0.3107

Epoch 58/100
35/35 [==============================] - 6s 175ms/step - loss: 0.3229 - accuracy: 0.8955 - val_loss: 0.5270 - val_accuracy: 0.8250
Epoch 59/100
35/35 [==============================] - 6s 179ms/step - loss: 0.3040 - accuracy: 0.9000 - val_loss: 0.4934 - val_accuracy: 0.8357
Epoch 60/100
35/35 [==============================] - 7s 192ms/step - loss: 0.3212 - accuracy: 0.9036 - val_loss: 0.5220 - val_accuracy: 0.8429
Epoch 61/100
35/35 [==============================] - 9s 247ms/step - loss: 0.2910 - accuracy: 0.9161 - val_loss: 0.5185 - val_accuracy: 0.8214
Epoch 62/100
35/35 [==============================] - 6s 181ms/step - loss: 0.2659 - accuracy: 0.9170 - val_loss: 0.4914 - val_accuracy: 0.8179
Epoch 63/100
35/35 [==============================] - 6s 179ms/step - loss: 0.3145 - accuracy: 0.8893 - val_loss: 0.7524 - val_accuracy: 0.7714
Epoch 64/100
35/35 [==============================] - 6s 176ms/step - loss: 0.5017 - accuracy: 0.8286 - val_loss: 0.6419 - val_accuracy:

In [158]:
model4.evaluate(test_x, test_y)

11/11 [==============================] - 1s 46ms/step - loss: 0.5845 - accuracy: 0.8400


[0.5844595432281494, 0.8399999737739563]